<a href="https://colab.research.google.com/github/nkinsman16/HW-Week-3-BasicCleaning_R/blob/main/BasicCleaning_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning: recurring coding strategies

Imagine we have this data:

In [ ]:
IRdisplay::display_html('<iframe width="700" height="300" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vR-ubcCBaveg-58jcVmbErpO5kZswjFyHN5YlB8tB1a8B4fzU4sqZ08jkOKx4kBz1qtDNkJJWH8vBYF/pubhtml?gid=0&single=true"></iframe>')

And you need to create a cleaner version:

In [ ]:
IRdisplay::display_html('<iframe width="700" height="300" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vR-ubcCBaveg-58jcVmbErpO5kZswjFyHN5YlB8tB1a8B4fzU4sqZ08jkOKx4kBz1qtDNkJJWH8vBYF/pubhtml?gid=2024244899&single=true"></iframe>')

The actual cleaning plan starts after exploration, and the  strategies to recover the true value of the cell can be one or several of these:

* **keep** the columns or rows needed
* **replace** the wrong value for good value.
* **delete** the wrong value.
* **extract** the good value.
* **split** the cell contents


**It is also recommended to clean the column names before the cell contents.**

Notice the **column names** have lightblue background,  and the **contents** have a grey background. I have also colored in red the column names that may serve as key columns, the unique identifiers.


## 1. Some REGEX

REGEX is **complex**, but let's share some _patterns_ that we will often use.

* This [a-zA-Z] represents any character of the alphabet (based in latin alphabet).
* This [^a-zA-Z] represents any characters *outside* the alphabet. The [^] means "the opposite" here. You can use for other purposes (see below).
* This \w is not the same as [a-zA-Z], \w can be equal to [0-9a-zA-Z\s_], that is, it includes numbers, spaces (\s) and underscore (_).
* Then, \W is the opposite of \w
* Similarly, \d is equal to [0-9], and \D is the opposite.
* Some other relevant symbols are:
  - The dot (**.**), representing any character.
  - The plus (**+**), here \d+ represents one or more digits.
  - The asterisk (__*__) here [a-z]* represents zero or more lowercase letters.
  - The symbols **^** and **\$** are also very relevant. Together the represent a whole string, that is,  ^\d[a-z]$ means the string starts with a digit and ends with a lowercase letter.

  You will need these when exploring and implementing the cleaning.

## 2. The Data Types

Imagine you CAN NOT see all the data. In that case we can use some code.

In [ ]:
# the link as CSV
linkToData="https://docs.google.com/spreadsheets/d/e/2PACX-1vR-ubcCBaveg-58jcVmbErpO5kZswjFyHN5YlB8tB1a8B4fzU4sqZ08jkOKx4kBz1qtDNkJJWH8vBYF/pub?gid=0&single=true&output=csv"

Read the data:

In [ ]:
dirty=read.csv(linkToData)

I recommend you do this first:

In [ ]:
str(dirty)

You should see the column names, but you do not ...because the column names are in the wrong place:

In [ ]:
#using 'head'
head(dirty)

Using _head_ and _tail_ is important:

In [ ]:
tail(dirty)

Notice empty cells have not been considered "NA". We could write instead:

In [ ]:
dirty=read.csv(linkToData,na.strings = c(""))
dirty

We are now in a situation when we beliveve we have a better idea of what rows/columns are needed.

# Cleaning Steps

## A. Keeping columns needed

Pay attention to column names:
- They should be on the top.
- They should have no spaces anywhere
- They should have no special characters
- They should have no start with numeric values
- They should be readable (short, self-explanatory)
- When needed, use comments to explain their meaning.

Currently the column names are somewhere they should not:

---



In [ ]:
#current names:
names(dirty)

In [ ]:
# They are in the contents
as.list(dirty[1,]) # always use list

In [ ]:
# renaming
colnames(dirty) <- as.list(dirty[1, ])

# remove first row (now it's the header)
dirty <- dirty[-1, ]

## see new column names
dirty

This data has columns with no values at all, even the column name is missing, we can use that to our advantage:

In [ ]:
names(dirty)[names(dirty)!='NA']

In [ ]:
# then
# using index labels
dirty <-   dirty[, names(dirty)[names(dirty)!='NA']]
# so
dirty

## B. Keeping rows needed on the contents

### B.1 Filtering using index positions

In [ ]:
dirty[1:6,] # this may seem confusing

In [ ]:
head(dirty,-2) # this works.. easy to understand?

### B.2 Filtering using a column with missing values:

In [ ]:
dirty[!is.na(dirty$identification2),]

Let's keep the first option:

In [ ]:
dirty= dirty[1:6,]

## C. Exploring cell contents

Identify which are textual, numerical, or categorical.

In [ ]:
str(dirty)



* Columns **identification1** and **identification2** are *textual*.
* The columns from **var1** to **var@3** are all *numerical*. But if the type is _chr_ the column has been read as strings.
* Column **category** is *categorical*. Keep in mind that categorical types will NEVER be recognised as such by default when read from a CSV. They will always be understood as text (_chr_).

The **column names** are always *strings*.


### C.2.1. **Exploring TEXT columns**

When data is textual, you need to explore the cells to verify all the characters are part of the **alphabet**.


Let's see how to use R:

In [ ]:
# show me the cells that have a character outside the alphabet
dirty$identification[grepl("[^a-zA-Z]", dirty$identification)]


United Kingdom is not dirty. But the space is outside the alphabet. What about:

In [ ]:
dirty[grepl("\\W", dirty$identification), 'identification']

or...

In [ ]:
# this looks ok
dirty$identification[grep("[^a-zA-Z\\s]",dirty$identification,perl = T)]

### C.2.2. **Exploring NUMBERS**

If numbers are recognised as so, there is no cleaning needed. But if not, it means it has been recognised as text, then we use the regex **\d** (and its variations):

In [ ]:
dirty$var1[grep("\\D",dirty$var1)]

In [ ]:
dirty[,'var 2'][grep("\\D",dirty$'var 2')]

In [ ]:
### remember you do not want this:
# dirty[,'var 2'][grep("\\D",dirty$'var 2')] |>as.numeric()

Notice I need to use **[  ]** to access the variables with dirty names (space between words, and the **@** special character). That is why you clean the column names first:

In [ ]:
dirty[,'var@3'][grep("\\D",dirty$'var@3',perl=T)]

There are cells with good values, but other values can not be kept. Use **\D** with care, numbers are complex. So I prefer something like this:

In [ ]:
dirty[,'var@3'][grep("[^\\d+\\.*\\d*]", dirty$'var@3', perl=T,invert = F)]

### C.2.3. **Exploring CATEGORICAL columns**

Just prepare a fequency table to see if categories are well defined.

In [ ]:
table(dirty$category)

### C.2.4. Exploring Column names

Here, we want to see what is not right.

In [ ]:
# names that have characters different than numbers or alphabet
colnames(dirty)[grepl("[^0-9a-zA-Z]", colnames(dirty))]

## C. Cleaning

As mentioned, cleaning may mean:

a. Making bad characters disappear.

b. Keeping good characters.


Let's start with the _column names_:

### C.1 Cleaning column names

How can you say: if "a space" or a "weird character", disappear? (that is, *replace* by *nothing*)


In [ ]:
# option 1
gsub("\\W",'',names(dirty) , perl=T )

In [ ]:
# option 2
gsub("[^\\w]",'',names(dirty), perl=T )

In [ ]:
# # option 3
gsub("[^0-9a-zA-Z]",'',names(dirty), perl=T )

Choose any and make the change:

In [ ]:
names(dirty)=gsub("[^0-9a-zA-Z]",'',names(dirty))
dirty

Be preventive about leading and trailing spaces:

In [ ]:
colnames(dirty) == trimws(colnames(dirty),whitespace = "[\\h\\v]")

The column names were cleaned by **Making bad characters disappear** 🙂

### C.2. Cleaning TEXT columns

Let's check the **identification** column:

In [ ]:
dirty$identification[grep("[^a-zA-Z\\s]",dirty$identification,perl = T)]

Not all cells have characters that are not in the alphabet;BUT THIS TIME, The **only** problem here is the brackets.

Then:

* Option 1: Whatever inside brackets (including the brackets) has to go!

In [ ]:
gsub("\\[.*\\]",'',dirty$identification,perl = T)

* Option 2: Splitting

In [ ]:
sapply(strsplit(dirty$identification, split = '[', fixed = TRUE), `[`, 1)

When you are satisfied, make the change:

In [ ]:
dirty$identification <- gsub("\\[.*\\]", "", dirty$identification)
dirty

The **splitting** option seems very convenient for **identification2**:

In [ ]:
sapply(strsplit(dirty$identification2, split = ',', fixed = TRUE), `[`, 2)

If this is OK, then:

In [ ]:
dirty$identification2 <- sapply(strsplit(dirty$identification2, split = ',', fixed = TRUE), `[`, 2)
dirty

Be preventive about leading and trailing spaces:

In [ ]:
dirty$identification == trimws(dirty$identification)

Check!

In [ ]:
dirty$identification

Then:

In [ ]:
dirty$identification <- trimws(dirty$identification)

In [ ]:
#verifying:
dirty$identification == trimws(dirty$identification)

The presence of ortographic symbols might complicate things. What about?

In [ ]:
## you may need to install this:
# install.packages(string)

In [ ]:
# load the stringi package
library(stringi)

# remove accents and special orthographic characters from 'identification'
stri_trans_general(dirty$identification, "Latin-ASCII")

In [ ]:
# then

dirty$identification <- stringi::stri_trans_general(dirty$identification, "Latin-ASCII")

### C.3. Cleaning the CAT column

We had this:

In [ ]:
table(dirty$category)

You can conclude that the **a** is wrong, it should be **A**.

In [ ]:
#what about:
gsub('a','A', dirty$category,fixed=T)

That changed **Ba** to **BA**!

In [ ]:
## maybe
## ^: start of string
## $: end  of string
gsub('^a$','A', dirty$category)

In [ ]:
#then
dirty$category <- gsub("^a$", "A", dirty$category)
dirty


### C.4. Cleaning NUM columns

From the previous exploration, we know some issues in these columns. Let's go step by step:

In [ ]:
gsub(',','',dirty$var1)


Then,

In [ ]:
dirty$var1=gsub(',','',dirty$var1)
dirty


The **var2** is more complicated.

In [ ]:
dirty$var2

Let me create a NEW variable as a flag:

In [ ]:
# save where you have the issue
dirty$var2_temp=grepl("\\'|k",dirty$var2,fixed=F)
dirty

In [ ]:
## now replace
dirty$var2 <- gsub("'|k|\\s", "", dirty$var2)
dirty

The last value was right. The other ones were simplified (lack '000'). Let's put the '000' back!

In [ ]:
# just adding 000 where needed
ifelse(dirty$var2_temp,paste0(dirty$var2,'000'),dirty$var2)


In [ ]:
# then
dirty$var2=ifelse(dirty$var2_temp,paste0(dirty$var2,'000'),dirty$var2)
# and delete this
dirty$var2_temp=NULL
# result so far
dirty


The **var3** can be solved like this:

In [ ]:
dirty['var3']=gsub("\\$|\\s",'',dirty$var3)
dirty

## D. Coding missing values:


Wrong missing values representation should be replaced with care. Do it according to the data type.

Then, let's start with the **categorical** column:

In [ ]:
badCats=grep('\\W+',dirty$category,value = T)
badCats

Let's go for the **numerical** cases:

In [ ]:
badNums1 <- dirty$var1[!grepl("\\d+.*\\d*", dirty$var1)]
badNums1

In [ ]:
badNums2 <- dirty$var2[!grepl("\\d+.*\\d*", dirty$var2)]
badNums2

In [ ]:
badNums3 <- dirty$var3[!grepl("\\d+.*\\d*", dirty$var3)]
badNums3

In [ ]:
unique(c(badCats, badNums1, badNums2, badNums3))

Or we can create a function:

In [ ]:
 detect_WrongNumber <- function(x) {
  x[!grepl("\\d+.*\\d*", x)]
  }


bads <- lapply(dirty[, c("var1", "var2", "var3")], detect_WrongNumber)
all_bads <- unique(unlist(bads))
all_bads <- union(all_bads, badCats)
all_bads



Let's recode all the **all_bads** into **NA**:

In [ ]:
# Let's recode all the **all_bads** into **NA**:
dirty[] <- data.frame(lapply(dirty, function(x) replace(x, x %in% all_bads, NA)))

Are we missing something?

## E. The Resetting of indexes

After many changes, you have this:

In [ ]:
dirty

In general, you want to be sure the row index starts in 'one' and continues with consecutive values. Just do this:

In [ ]:
rownames(dirty)=NULL
print(dirty)

In [ ]:
dirty

# SAVING the CLEAN data

In [ ]:
write.csv(dirty,"nowClean.csv")


In [ ]:
nowClean=dirty[,]


folder <- "dataCleaned"

# Check if the folder exists
if (!dir.exists(folder)) {
  # Create the folder
  dir.create(folder)
  write.csv(nowClean,file.path(folder,"nowClean.csv"))

} else {
  write.csv(nowClean,file.path(folder,"nowClean.csv"))}
